In [33]:
import cv2
import mediapipe as mp
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from PIL import Image, ImageTk
import numpy as np

# Felület inicializálása
root = tk.Tk()
root.title("Hand&Gesture")

# Sötét téma beállítása
root.configure(bg='#000000')
style = ttk.Style()
style.theme_use('clam')
style.configure('.', background='#000000', foreground='#FFFFFF', fieldbackground='#2E2E2E')
style.configure('TButton', background='#000000', foreground='#FFFFFF', relief="flat")
style.configure('TCheckbutton', background='#000000', foreground='#FFFFFF')
style.configure('TLabel', background='#000000', foreground='#FFFFFF')

# Logó hozzáadása
try:
    # Kép betöltése és átméretezése
    logo_image = Image.open("logo.png")
    logo_image = logo_image.resize((200, 200), Image.Resampling.LANCZOS)
    logo_photo = ImageTk.PhotoImage(logo_image)

    logo_label = tk.Label(root, image=logo_photo, bg='#000000')
    logo_label.image = logo_photo
    logo_label.pack(pady=10)
except Exception as e:
    messagebox.showwarning("Figyelem", f"Nem sikerült betölteni a logót: {e}")

# Kamera inicializálása
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    messagebox.showerror("Hiba", "Nem található kamera!")
    root.destroy()
    exit()

# MediaPipe modulok
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_face_detection = mp.solutions.face_detection

# Beállítások
hand_detection_enabled = tk.BooleanVar(value=True)
face_blur_enabled = tk.BooleanVar(value=True)

def start_processing():
    """A kamera és a feldolgozás elindítása."""
    # Gombok letiltása
    hand_detection_check.config(state="disabled")
    face_blur_check.config(state="disabled")

    with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands, \
         mp_face_detection.FaceDetection(min_detection_confidence=0.7) as face_detection:
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                print("Hiba: Nem sikerült a képkockát olvasni!")
                break

            # BGR színformátumból RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Kéz és arcfelismerés végrehajtása
            hand_results = hands.process(image) if hand_detection_enabled.get() else None
            face_results = face_detection.process(image) if face_blur_enabled.get() else None

            image.flags.writeable = True
            frame = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            total_score = 0  # Az összes felemelt ujj értéke

            # Arc homályosítása
            if face_results and face_results.detections:
                for detection in face_results.detections:
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = frame.shape
                    x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

                    # Maszk létrehozása az arc követéséhez
                    mask = np.zeros((ih, iw), dtype=np.uint8)
                    center_coordinates = (x + w // 2, y + h // 2)
                    axes = (w // 2, h // 2)
                    cv2.ellipse(mask, center_coordinates, axes, 0, 0, 360, 255, -1)

                    # Az arc területének homályosítása
                    blurred_frame = cv2.GaussianBlur(frame, (51, 51), 30)
                    frame = np.where(mask[..., None] == 255, blurred_frame, frame)

            # Kézjelek feldolgozása
            if hand_results and hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    # Ujjtippek és értékek
                    fingertips = [
                        (mp_hands.HandLandmark.INDEX_FINGER_TIP, 2),
                        (mp_hands.HandLandmark.MIDDLE_FINGER_TIP, 4),
                        (mp_hands.HandLandmark.RING_FINGER_TIP, 8),
                        (mp_hands.HandLandmark.PINKY_TIP, 16),
                    ]

                    hand_score = 0  # Egy kéz pontszáma

                    # Hüvelykujj ellenőrzése
                    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                    thumb_ip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]
                    thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
                    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]

                    # A hüvelykujj akkor van leengedve, ha a TIP alacsonyabban van az MCP-hez képest
                    if thumb_tip.x < thumb_ip.x and thumb_tip.y < index_mcp.y:
                        cv2.putText(frame, "1", (int(thumb_tip.x * frame.shape[1]), int(thumb_tip.y * frame.shape[0]) - 20),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        hand_score += 1

                    # Más ujjak ellenőrzése
                    for tip, value in fingertips:
                        fingertip = hand_landmarks.landmark[tip]
                        base_joint = hand_landmarks.landmark[tip - 2]  # Az ujj alapízülete
                        x, y = int(fingertip.x * frame.shape[1]), int(fingertip.y * frame.shape[0])

                        # Ha az ujj felfelé áll (tip alacsonyabban van, mint az alapízület)
                        if fingertip.y < base_joint.y:
                            hand_score += value
                            cv2.putText(frame, str(value), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    total_score += hand_score

                    # Kézrajz megjelenítése
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Összpontszám megjelenítése a képernyőn
            cv2.putText(frame, f"Total: {total_score}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

            # Kép megjelenítése
            cv2.imshow("Hand Gesture and Face Blur", frame)

            # Kilépés q billentyű lenyomásával VAGY ablak bezárásával
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            if cv2.getWindowProperty("Hand Gesture and Face Blur", cv2.WND_PROP_VISIBLE) < 1:
                break

    # Kamera leállítása és ablak bezárása
    cap.release()
    cv2.destroyAllWindows()

def exit_app():
    """Kilépés az alkalmazásból."""
    cap.release()
    root.destroy()

# GUI elemek
ttk.Label(root, text="Funkciók testreszabása:", font=("Helvetica", 16)).pack(pady=10)
hand_detection_check = ttk.Checkbutton(root, text="Ujjak lekövetése", variable=hand_detection_enabled)
hand_detection_check.pack(pady=5)
face_blur_check = ttk.Checkbutton(root, text="Arc kitakarása", variable=face_blur_enabled)
face_blur_check.pack(pady=5)
ttk.Button(root, text="Start", command=start_processing).pack(pady=10)
ttk.Button(root, text="Exit", command=exit_app).pack(pady=5)

# Fő eseményhurok
root.mainloop()
